## Class Tutorial

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pickle 
from astropy.io import fits
%matplotlib widget

We use a class to define several functions in a self-consistent way repeatedly and be able to use them in different notebooks without rewriting them.

### Class Objects
Class objects support two kind of operations: *attribute references* and *instantiation*. **Attribute references** use the standard syntax in python (obj.name).

In [22]:
# Example:
class MyClass:
    """A simple example class"""
    internal_variable = 12345

    def print_hello(self):
        return 'Hello World'

# To retrieve the value of the internal variable we use:
print(MyClass.internal_variable)
# To use the function of the class we use
MyClass.print_hello(MyClass)

12345


'Hello World'

The argument $\texttt{self}$ of the class means that the function requires the class itself as an argument.

**Class instantiation** uses function notation. We can create a new instance of the class and assign his object to a local variable.

In [23]:
first_class = MyClass()
second_class = MyClass()
# We have two objects of type MyClass with all the attributes and functions
# of the original class but will work independently

second_class.internal_variable = 67890
print('first_class internal variable value:', first_class.internal_variable)
print('second_class internal variable value:', second_class.internal_variable)

first_class internal variable value: 12345
second_class internal variable value: 67890


A variable inside a class takes the name of **attribute** while functions are called **methods**. When calling a method of an instance object you don0t need to specify the self argument

In [24]:
MyClass.print_hello(MyClass) #Here I need the self argument
second_class.print_hello() #Here I don't need it, notice the difference

'Hello World'

### Class Initialization
Classes support a special kind of method that is automatically called **every time** you create a new class instance. This inizialization method is defined by the name ```__init__```. When we create a new object using our class, the init method is automatically invoked. In the following example init requires two arguments other than the self one, so we have to provide them to avoid an error 

In [25]:
class MySecondClass:
    def __init__(self, x_input, y_input):
        self.x_pos = x_input
        self.y_pos = y_input

#new_class = MySecondClass() -> WRONG! No arguments
new_class = MySecondClass(258, 76) # Correct

If we change the class, we have to make a new instance, the previous one won't be updated.

In [26]:
class MySecondClass:
    def __init__(self, x_input, y_input):
        self.x_pos = x_input
        self.y_pos = y_input

    def print_position(self):
        print('Attributes called from a class method', self.x_pos, self.y_pos)

    def change_position(self, x_new, y_new):
        self.x_pos = x_new
        self.y_pos = y_new

newest_class = MySecondClass(260, 76)
newest_class.print_position()

print('attributes called from instance object:', new_class.x_pos, new_class.y_pos)

Attributes called from a class method 260 76
attributes called from instance object: 258 76


We can write a method to change one or more attributes. The method must get ```self``` as the argument, and the attributes to be changed must be referenced as ```self```

In [27]:
class MySecondClass:

    def change_position(self, x_new, y_new):
        self.x_pos = x_new
        self.y_pos = y_new

    """def wrong_method(x_new, y_new):
        x_pos = x_new
        y_pos = y_new"""
    
    """All the methods called after the change_position will use the updated
    values for self.x_pos and self.y_pos"""

new_class = MySecondClass()
new_class.change_position(789, 123)
print(new_class.x_pos, new_class.y_pos)

789 123


## Aperture Photometry Class
Let's make our class to perform aperture photometry with different apertures and on several stars effortlessly. The complete code will be exported in a .py file for further use.

### Initialization
As this is a work in progress, we will call our class ```TemporaryAperturePhotometry``` for now. We provide some constants that will be the same regardless of the star under analysis (given that we use the same frames).


In [28]:
class TemporaryAperturePhotometry:
    def __init__(self):
        
        self.data_path = './group10_WASP-135_20190803/'

        # Constants from HEADER file
        self.readout_noise = 7.10 # [e] photoelectrons
        self.gain = 1.91 # [e] photoelectrons

        # Computed in 01 - Bias Analysis.ipynb
        self.bias_std = 1.33 # [e] = photoelectrons
        self.median_bias = pickle.load(open('./median_bias.p', 'rb'))
        self.median_bias_error_distribution = pickle.load(open('./median_bias_error.p', 'rb'))
        # We chose to use the specific value in lecture 01
        self.median_bias_error_value = pickle.load(open('./median_bias_error_value.p', 'rb'))

        # Computed in 02 - Flat Analysis.ipynb
        self.median_normalized_flat = pickle.load(open('./median_normalized_flat.p', 'rb'))
        self.median_normalized_flat_error = pickle.load(open('./median_normalized_flat_errors.p'))

        # Computed in 03 - Science Analysis.ipynb
        self.science_path = self.data_path + 'science/'
        self.science_list = np.genfromtxt(self.science_path + 'science.list', dtype = str)
        self.science_size = len(self.science_list)

